In [1]:
#匯入所需模組

import os
import pandas as pd
import numpy as np
import cv2
import PIL.Image
from keras.preprocessing import image
from keras.utils import np_utils
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.applications import *
from keras.optimizers import Adam
from keras.layers import Dense, Activation, Flatten
from keras.layers import Input,Dropout
from keras.models import Model

Using TensorFlow backend.


In [2]:
label_list=list()
img_name_list=list()
fp=open('./train.csv', "r")
for line in iter(fp):
    img_name_list.append(line[:-3])
    if line[-2:-1]=='A':
        label_list.append(0)
    elif line[-2:-1]=='B':
        label_list.append(1)
    elif line[-2:-1]=='C':
        label_list.append(2)
fp.close()

In [3]:
#讀進訓練集並做正規化
path='./C1-P1_Train/'
img_data_list=[]

for img in img_name_list:
    img_path=path+img
    IMG=image.load_img(img_path,target_size=(224,224))
    im=image.img_to_array(IMG)
#     im=np.expand_dims(im,axis=0)
    im=im/255.0
    img_data_list.append(im)

img_data = np.array(img_data_list)

print (img_data.shape)  
# img_data=np.rollaxis(img_data,1,0)
# print (img_data.shape) 
# img_data=img_data[0]
# print (img_data.shape)  

(5600, 224, 224, 3)


In [4]:
#設定標籤值
num_classes=3
Y=np_utils.to_categorical(label_list,num_classes)
x,y=shuffle(img_data,Y,random_state=2)

#訓練集80%、驗證集20%
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.15,random_state=2)

In [9]:
# from keras.models import Sequential
# from keras.layers import Dense, Activation, Dropout, Flatten
# from keras.layers import Conv2D
# from keras.layers import MaxPooling2D

# input_shape = (224, 224, 3)

# model = Sequential([
#     Conv2D(64, (3, 3), input_shape=input_shape, padding='same',
#            activation='relu'),
#     Conv2D(64, (3, 3), activation='relu', padding='same'),
#     MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
#     Conv2D(128, (3, 3), activation='relu', padding='same'),
#     Conv2D(128, (3, 3), activation='relu', padding='same',),
#     MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
#     Conv2D(256, (3, 3), activation='relu', padding='same',),
#     Conv2D(256, (3, 3), activation='relu', padding='same',),
#     Conv2D(256, (3, 3), activation='relu', padding='same',),
#     MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
#     Conv2D(512, (3, 3), activation='relu', padding='same',),
#     Conv2D(512, (3, 3), activation='relu', padding='same',),
#     Conv2D(512, (3, 3), activation='relu', padding='same',),
#     MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
#     Conv2D(512, (3, 3), activation='relu', padding='same',),
#     Conv2D(512, (3, 3), activation='relu', padding='same',),
#     Conv2D(512, (3, 3), activation='relu', padding='same',),
#     MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
#     Flatten(),
# #     Dense(4096, activation='relu'),
# #     Dense(1024, activation='softmax'),
#     Dropout(0.5),
#     Dense(3, activation='softmax')
# ])

# model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=1e-5),metrics=['accuracy'])
# model.fit(x_train,y_train,validation_data=(x_test,y_test),batch_size=16,epochs=60,verbose=1)

In [5]:
#使用InceptionV3進行模型訓練
image_input=Input(shape=(224,224,3))
model=VGG16(include_top=False, weights='imagenet', input_shape=(224,224,3),input_tensor=image_input)
# model=VGG16(include_top=False, weights='imagenet', input_shape=(224,224,3),input_tensor=image_input,pooling='max')

last_layer = model.output
last_layer=Flatten()(last_layer)
last_layer=Dropout(0.5)(last_layer)
out = Dense(num_classes, activation='softmax', name='softmax')(last_layer)
custom_V3_model = Model(image_input, out)
# custom_V3_model.summary()

custom_V3_model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=1e-5),metrics=['accuracy'])
custom_V3_model.fit(x_train,y_train,validation_data=(x_test,y_test),batch_size=16,epochs=30,verbose=1)







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 4760 samples, validate on 840 samples
Epoch 1/30
4760/4760 [==============================] - 153s 32ms/step - loss: 0.9582 - acc: 0.5477 - val_loss: 0.6509 - val_acc: 0.7179
Epoch 2/30
4760/4760 [==============================] - 134s 28ms/step - loss: 0.6853 - acc: 0.7034 - val_loss: 0.6232 - val_acc: 0.7190
Epoch 3/30
4760/4760 [==============================] - 134s 28ms/step - loss: 0.5885 - acc: 0.7489 - val_loss: 0.6106 - val_acc: 0.7452
Epoch 4/30
4760/4760 [==============================] - 134s 28ms/step - loss: 0.5258 - acc: 0.7803 - val_loss: 0.5953 - val_acc: 0.7595
Epoch 5/30
4760/4760 [==============================] - 134s 28ms/step - loss: 0.4643 - acc: 0.8088 - val_loss: 0.6114 - val_acc: 0.7536
Epoch 6/30
4760/4760 [==============================

In [8]:
prediction=custom_V3_model.predict(x_test)
predict=np.argmax(prediction,axis=1)

In [9]:
predict

array([1, 1, 1, 2, 0, 2, 0, 2, 2, 2, 1, 2, 0, 2, 2, 2, 1, 0, 1, 1, 2, 0,
       1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 2, 2, 2, 1, 1, 0, 2, 1, 2, 0, 1, 1,
       0, 1, 0, 0, 0, 1, 2, 2, 0, 1, 0, 2, 0, 1, 1, 0, 1, 2, 0, 2, 1, 1,
       2, 0, 0, 2, 2, 2, 0, 0, 1, 0, 0, 0, 1, 1, 1, 2, 0, 0, 2, 2, 1, 0,
       0, 0, 1, 0, 0, 1, 1, 0, 1, 2, 0, 0, 1, 0, 2, 1, 0, 0, 1, 1, 2, 1,
       1, 1, 2, 0, 2, 1, 0, 1, 2, 0, 2, 2, 0, 2, 0, 0, 1, 2, 2, 1, 1, 1,
       1, 2, 0, 1, 0, 1, 2, 1, 2, 0, 0, 2, 1, 2, 1, 0, 1, 0, 0, 0, 0, 0,
       1, 2, 0, 1, 1, 0, 0, 1, 1, 1, 2, 0, 1, 1, 1, 0, 0, 2, 1, 0, 0, 1,
       0, 1, 2, 1, 0, 2, 0, 0, 0, 2, 1, 2, 2, 1, 2, 0, 0, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 2, 1, 0, 0, 2, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 2, 1, 1, 0, 1, 1, 1, 2, 1, 0, 2, 2, 2, 0, 2, 1, 0, 0, 1,
       2, 1, 2, 1, 1, 1, 1, 1, 0, 1, 0, 2, 2, 2, 0, 1, 1, 1, 1, 0, 2, 1,
       1, 0, 0, 2, 1, 1, 1, 1, 2, 0, 0, 1, 0, 0, 2, 0, 1, 0, 2, 1, 0, 0,
       0, 2, 2, 1, 0, 0, 1, 2, 0, 0, 2, 0, 0, 0, 1,

In [16]:
from sklearn.metrics import recall_score

y_true=np.argmax(y_test,axis=1)
recall_score(y_true, predict, average='weighted')

0.7571428571428571

In [10]:
model.save('vgg16.h5')

In [6]:
custom_V3_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________